<a href="https://colab.research.google.com/github/kalai2315/Langchain_for_AgentiAI/blob/main/Customer_review_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini project 3: Building a Customer Review Analyst

**Routing Chains in LCEL**

A Routing Chain is a conditional logic mechanism that dynamically chooses which chain to run based on some property of the input or intermediate output.

In [ ]:

%pip install -U --quiet langchain langchain-google-genai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:

import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API Key: ")

Enter your Google API Key: ··········


In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-latest")

In [ ]:

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser

In [ ]:
classifier_prompt = ChatPromptTemplate.from_template(
    """ Given the user instruction below for analysing customer review
    classify it as only one of the categories:
      - summarize
      - sentimental
      -email

    Do not respond with more than one word

    Instruction: {instruction}
"""

)

classifier_chain = classifier_prompt | llm | StrOutputParser()

In [ ]:
summary_prompt = ChatPromptTemplate.from_template(
    """ Act as a customer review analyst, given the following customer review:
        generate short summary(max 2 lines) of the review

        customer review: {review}
    """)

summary_chain = summary_prompt | llm | StrOutputParser()


In [ ]:
sentiment_prompt = ChatPromptTemplate.from_template(
    """ Act as a customer review analyst, given the following customer review:
        find out the sentiment of the review, either positive, negative or neutral
        return only one word, either positive, negative or neutral

        customer review: {review}
    """)

sentiment_chain = sentiment_prompt | llm | StrOutputParser()


In [ ]:
email_prompt = ChatPromptTemplate.from_template(
    """ Act as a customer review analyst, given the following customer review and sentiment
        generate an email response to the customer based on the following condition.
        If the sentiment is positive or neutral, thank them for the review
        If the sentiment is negative, apologize and suggest that they can improve their review

        sentiment: {sentiment}

        customer review: {review}
        sentiment: {sentiment}
    """)

email_chain = email_prompt | llm | StrOutputParser()


In [ ]:
def default_answer(query):
  return "sorry instructions are not the defined intents"

In [ ]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "summarize" in x["topic"].lower(), summary_chain),
    (lambda x: "sentimental" in x["topic"].lower(), sentiment_chain),
    (lambda x: "email" in x["topic"].lower(), email_chain),
    default_answer
)

In [ ]:
full_chain = ({"topic" : classifier_chain,
               "instructions": lambda input_prompt: input_prompt.get("instruction"),
               "review": lambda input_prompt: input_prompt.get("review"),
               "sentiment": lambda input_prompt: input_prompt.get("sentiment"),
               }
              | branch)



In [ ]:
sample_review = """
          Required a stylish lamp for my office space, and this particular one came with added storage at a reasonable price.
          The delivery was surprisingly quick, arriving within just two days.
          However, the pull string for the lamp suffered damage during transit.
          To my relief, the company promptly dispatched a replacement, which arrived within a few days. Assembly was a breeze.
          Then, I encountered an issue with a missing component, but their support team responded swiftly and provided the missing part.
          It appears to be a commendable company that genuinely values its customers and the quality of its products.
"""



In [ ]:
summary = full_chain.invoke({"instruction": "Generate a summary for the given review",
                             "review": sample_review})
summary

'Stylish lamp with convenient storage arrived quickly, despite initial damage to the pull string and a missing component.  Excellent customer service ensured prompt replacements and resolution of all issues.'

In [ ]:
sentiment = full_chain.invoke({"instruction": "Find out the sentiment of the customer for the given review",
                             "review": sample_review})
sentiment

'Positive'

In [ ]:
email_response = full_chain.invoke({"instruction": "Write an email for the given customer review",
                             "review": sample_review,
                               "sentiment": sentiment})
print(email_response)

Subject: Thank you for your feedback!

Dear [Customer Name],

Thank you so much for taking the time to leave such a positive review! We're thrilled to hear you love your new lamp and appreciate your kind words about our product, delivery speed, and customer support.

We're especially glad that we were able to resolve the issues you experienced with the pull string and missing component so quickly.  We strive to provide excellent customer service and are delighted that we met your expectations in this instance.

Your feedback is invaluable to us, and we appreciate you sharing your experience.  We hope you enjoy your stylish and functional new lamp for many years to come!

Sincerely,

The [Company Name] Team
